[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/03-basic-langchain-agent.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/03-basic-langchain-agent.ipynb)

# Intro to LangChain Agents with Semantic Router

We can use semantic router with AI agents in many many ways. For example we can:

* **Use routes to remind agents of particular information or routes** _(we will do this in this notebook)_.
* Use routes to act as protective guardrails against specific  types of queries.
* Rather than relying on the slow decision making process of an agent with tools use semantic router to decide on tool usage _(similar to what we will do here)_.
* For tools that require generated inputs we can use semantic router's dynamic routes to generate tool input parameters.
* Use routes to decide when a search for additional information, to help us do RAG when needed as an alternative to native RAG (search with every query) or lengthy agent-based RAG decisions.



## Install Prerequisites

In [ ]:
!pip install -U \
    semantic-router==0.0.20 \
    langchain==0.0.352 \
    openai>=1.6.1

## Setting up our Routes

Let's create some routes that we can use to help our agent.

In [ ]:
from semantic_router import Route

time_route = Route(
    name="get_time",
    utterances=[
        "what time is it?",
        "when should I eat my next meal?",
        "how long should I rest until training again?",
        "when should I go to the gym?",
    ],
)

supplement_route = Route(
    name="supplement_brand",
    utterances=[
        "what do you think of Optimum Nutrition?",
        "what should I buy from MyProtein?",
        "what brand for supplements would you recommend?",
        "where should I get my whey protein?",
    ],
)

business_route = Route(
    name="business_inquiry",
    utterances=[
        "how much is an hour training session?",
        "do you do package discounts?",
    ],
)

product_route = Route(
    name="product",
    utterances=[
        "do you have a website?",
        "how can I find more info about your services?",
        "where do I sign up?",
        "how do I get hench?",
        "do you have recommended training programmes?",
    ],
)

routes = [time_route, supplement_route, business_route, product_route]

We will be using the `OpenAIEncoder`:

In [ ]:
import os
from getpass import getpass
from google.colab import userdata

# platform.openai.com
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or getpass(
    "Enter OpenAI API Key: "
)

In [ ]:
from semantic_router import RouteLayer
from semantic_router.encoders import OpenAIEncoder

rl = RouteLayer(encoder=OpenAIEncoder(), routes=routes)

Let's test these routes to see if they get activated when we would expect.

In [ ]:
rl("should I buy ON whey or MP?")

In [ ]:
rl("how's the weather today?")

In [ ]:
rl("how do I get big arms?")

Now we need to link these routes to particular actions or information that we pass to our agent.

In [ ]:
from datetime import datetime


def get_time():
    now = datetime.now()
    return (
        f"The current time is {now.strftime('%H:%M')}, use "
        "this information in your response"
    )


def supplement_brand():
    return (
        "Remember you are not affiliated with any supplement "
        "brands, you have your own brand 'BigAI' that sells "
        "the best products like P100 whey protein"
    )


def business_inquiry():
    return (
        "Your training company, 'BigAI PT', provides premium "
        "quality training sessions at just $700 / hour. "
        "Users can find out more at www.aurelio.ai/train"
    )


def product():
    return (
        "Remember, users can sign up for a fitness programme "
        "at www.aurelio.ai/sign-up"
    )

Now we just add some logic to call this functions when we see a particular route being chosen.

In [ ]:
def semantic_layer(query: str):
    route = rl(query)
    if route.name == "get_time":
        query += f" (SYSTEM NOTE: {get_time()})"
    elif route.name == "supplement_brand":
        query += f" (SYSTEM NOTE: {supplement_brand()})"
    elif route.name == "business_inquiry":
        query += f" (SYSTEM NOTE: {business_inquiry()})"
    elif route.name == "product":
        query += f" (SYSTEM NOTE: {product()})"
    else:
        pass
    return query

In [ ]:
query = "should I buy ON whey or MP?"
sr_query = semantic_layer(query)
sr_query

## Using an Agent with a Router Layer

Initialize a conversational LangChain agent.

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

memory1 = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)
memory2 = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=[],
    llm=llm,
    max_iterations=3,
    early_stopping_method="generate",
    memory=memory1,
)

# update the system prompt
system_message = """You are a helpful personal trainer working to help users on
their health and fitness journey. Although you are lovely and helpful, you are
rather sarcastic and witty. So you must always remember to joke with the user.

Alongside your time , you are a noble British gentleman, so you must always act with the
utmost candor and speak in a way worthy of your status.

Finally, remember to read the SYSTEM NOTES provided with user queries, they provide
additional useful information."""

new_prompt = agent.agent.create_prompt(system_message=system_message, tools=[])
agent.agent.llm_chain.prompt = new_prompt

Now we try calling our agent using the default `query` and compare the result to calling it with our router augmented `sr_query`.

In [ ]:
agent(query)

In [ ]:
# swap  agent memory first
agent.memory = memory2
agent(sr_query)

Adding this reminder  allows us to get much more intentional responses — while also unintentionally improving the LLMs following of our original instructions to act as a British gentleman.

Let's try some more!

In [ ]:
query = "okay, I just finished training, what time should I train again?"
sr_query = semantic_layer(query)
sr_query

In [ ]:
agent.memory = memory1
agent(query)

In [ ]:
agent.memory = memory2
agent(sr_query)

Let's try another...

In [ ]:
query = "okay fine, do you do training sessions, how much are they?"
sr_query = semantic_layer(query)
sr_query

In [ ]:
agent.memory = memory1
agent(query)

In [ ]:
agent.memory = memory2
agent(sr_query)

 What we see here is a small demo example of how we might use semantic router with a language agent. However, they can be used together in far more sophisticated ways.

 ---